In [117]:
import glob
import json
import os
import pandas as pd
import warnings
import sqlite3
warnings.filterwarnings("ignore")
DATA_FOLDER = '../data/raw'
zillow_files = [os.path.join(DATA_FOLDER, 'zillow',  file) 
             for file in os.listdir(DATA_FOLDER + "/zillow") 
             if file.endswith('.json')]
#chat GPT helped me with the following code
combined_data = []
for file in zillow_files:
  #found a stackoverflow how to read json files line by line
  with open(file, 'r') as f:
    for line in f:
      combined_data.append(json.loads(line))
    
    
#until here

# print(combined_data[0])
combined_df = pd.DataFrame.from_dict(combined_data)
print(type(combined_data[0]))
display(combined_df.columns)


<class 'dict'>


Index(['zpid', 'id', 'rawHomeStatusCd', 'marketingStatusSimplifiedCd',
       'imgSrc', 'hasImage', 'detailUrl', 'statusType', 'statusText',
       'countryCurrency', 'price', 'unformattedPrice', 'address',
       'addressStreet', 'addressCity', 'addressState', 'addressZipcode',
       'isUndisclosedAddress', 'beds', 'baths', 'area', 'latLong',
       'isZillowOwned', 'flexFieldText', 'flexFieldType', 'hdpData', 'isSaved',
       'isUserClaimingOwner', 'isUserConfirmedClaim', 'pgapt', 'sgapt',
       'zestimate', 'shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo',
       'isHomeRec', 'hasAdditionalAttributions', 'isFeaturedListing',
       'isShowcaseListing', 'list', 'relaxed', 'info6String', 'brokerName',
       'carouselPhotos', 'hasOpenHouse', 'openHouseStartDate',
       'openHouseEndDate', 'openHouseDescription', 'providerListingId',
       'builderName', 'isPropertyResultCDP', 'lotAreaString'],
      dtype='object')

### Now drop unnecessary columns

In [118]:
combined_cleaned = combined_df[['id', 'price', 'address', 
    'addressZipcode', 'addressState', 'beds', 'baths', 'area', 
    'zestimate', 'brokerName']]
# display(combined_cleaned.head())
# display(combined_cleaned.head())
# display(combined_cleaned[combined_cleaned['beds'] ==0])

I only want houses which are real (have at least 1 bedroom)

In [119]:
combined_real = combined_cleaned[combined_cleaned['beds'] > 0]
display(combined_real.info())
display(combined_real.head())

<class 'pandas.core.frame.DataFrame'>
Index: 767 entries, 0 to 819
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              767 non-null    object 
 1   price           767 non-null    object 
 2   address         767 non-null    object 
 3   addressZipcode  767 non-null    object 
 4   addressState    767 non-null    object 
 5   beds            767 non-null    float64
 6   baths           767 non-null    float64
 7   area            767 non-null    float64
 8   zestimate       589 non-null    float64
 9   brokerName      722 non-null    object 
dtypes: float64(4), object(6)
memory usage: 65.9+ KB


None

,id,price,address,addressZipcode,addressState,beds,baths,area,zestimate,brokerName
0,59150059,"$1,595,000","16 Atlantic St, South Boston, MA 02127",02127,MA,5.0,3.0,2300.0,1538400.0,Grove Property Group
1,2063017667,"$899,000","330 1/2 East St #1, Boston, MA 02127",02127,MA,4.0,2.0,1662.0,NaN,Gibson Sotheby's International Realty
2,63712077,"$850,000","534-538 Commonwealth Ave UNIT 7D, Boston, MA 0...",02215,MA,2.0,2.0,864.0,800100.0,Cornerstone Real Estate
3,318227332,"$939,000","183 D St UNIT 4, South Boston, MA 02127",02127,MA,2.0,2.0,1037.0,927800.0,Skyline Realty
5,190014785,"$899,000","22 Florian St, Roslindale, MA 02131",02131,MA,6.0,2.0,1959.0,872500.0,Here Realty LLC


Now all houses have non-null beds, baths, etc
Next is to strip the $ sign from price, add a town column, and convert to proper data types

In [120]:
# display(combined_real[combined_real['price'] == '779000+'])


In [121]:
combined_real['price']  = combined_real['price'].str.replace('$', '').str.replace(',','').str.replace('+', '')
combined_real['price'] = combined_real['price'].astype(float)
display(combined_real.info())

<class 'pandas.core.frame.DataFrame'>
Index: 767 entries, 0 to 819
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              767 non-null    object 
 1   price           767 non-null    float64
 2   address         767 non-null    object 
 3   addressZipcode  767 non-null    object 
 4   addressState    767 non-null    object 
 5   beds            767 non-null    float64
 6   baths           767 non-null    float64
 7   area            767 non-null    float64
 8   zestimate       589 non-null    float64
 9   brokerName      722 non-null    object 
dtypes: float64(5), object(5)
memory usage: 65.9+ KB


None

bedroom can only be an integer, so will cast it as an int16. ID is also an int

In [122]:
combined_real['beds'] = combined_real['beds'].astype('int16')
combined_real['id'] = combined_real['id'].astype('int64')
combined_real['town'] = combined_real['address'].str.split(',').str[1]
combined_real.drop_duplicates(subset='address', inplace=True)
display(combined_real.info())

<class 'pandas.core.frame.DataFrame'>
Index: 766 entries, 0 to 819
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              766 non-null    int64  
 1   price           766 non-null    float64
 2   address         766 non-null    object 
 3   addressZipcode  766 non-null    object 
 4   addressState    766 non-null    object 
 5   beds            766 non-null    int16  
 6   baths           766 non-null    float64
 7   area            766 non-null    float64
 8   zestimate       588 non-null    float64
 9   brokerName      721 non-null    object 
 10  town            766 non-null    object 
dtypes: float64(4), int16(1), int64(1), object(5)
memory usage: 67.3+ KB


None

In [124]:
%load_ext sql
%config SqlMagic.autocommit=True # for engines that do not support autommit

Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [125]:
%%sql sqlite:///../data/processed/home_prices.db --alias supermarket

UsageError: %%sql is a cell magic, but the cell body is empty. Did you mean the line magic %sql (single %)?
